In [1]:
import pandas as pd
import geopandas as gpd

# Medellin

In [9]:
# # Import the population data
# DANE_data = pd.read_csv('../data/input/table/DANE_2018_personas_manz.txt',low_memory=False)
# Import medellin blocks
blocks = gpd.read_file('../data/input/shape/Medellin_blocks_DANE_2018_new/Medellin_blocks_DANE_2018_new.shp')

In [ ]:
# Filter blocks' information

cols = ['COD_DANE_A','TP27_PERSO','DENSIDAD','TVIVIENDA','TP9_1_USO','TP9_2_1_MI','TP9_2_2_MI','TP9_2_3_MI','TP32_1_SEX','TP32_2_SEX',
        'TP34_1_EDA','TP34_2_EDA','TP34_3_EDA','TP34_4_EDA','TP34_5_EDA','TP34_6_EDA','TP34_7_EDA','TP34_8_EDA','TP34_9_EDA',
        'TP51PRIMAR','TP51SECUND','TP51SUPERI','TP51POSTGR','geometry']
blocks_m = blocks[cols].copy()
for i in cols[2:-1]:
    blocks_m[i] = blocks_m[i].astype(int)

blocks_m = blocks_m.rename(columns={'COD_DANE_A':'MANZ_CCNCT','TP27_PERSO':'pop','DENSIDAD':'rho','TVIVIENDA':'n_viv',
                                    'TP9_1_USO':'n_res','TP9_2_1_MI':'n_ind','TP9_2_2_MI':'n_com','TP9_2_3_MI':'n_ser',
                                    'TP32_1_SEX':'n_hom','TP32_2_SEX':'n_muj',
                                    'TP34_1_EDA':'n_00_09_y','TP34_2_EDA':'n_10_19_y','TP34_3_EDA':'n_20_29_y','TP34_4_EDA':'n_30_39_y','TP34_5_EDA':'n_40_49_y',
                                    'TP34_6_EDA':'n_50_59_y','TP34_7_EDA':'n_60_69_y','TP34_8_EDA':'n_70_79_y','TP34_9_EDA':'n_80_mo_y',
                                    'TP51PRIMAR':'n_edu_pri','TP51SECUND':'n_edu_sec','TP51SUPERI':'n_edu_sup','TP51POSTGR':'n_edu_pos'})
blocks_m = blocks_m.to_crs('epsg:32618')
blocks_m['rho'] = blocks_m['pop']/(blocks_m.area/1e4) # hab/hec
blocks_m = blocks_m.to_crs('epsg:4326')
blocks_m.to_file('../data/output/shape/blocks/medellin_blocks.shp')

In [ ]:
# Zona de analisis
zona = gpd.read_file('../data/input/shape/Poligonos/Alto_Alta.shp')
zona = zona.to_crs('epsg:4326')
zona = zona[['geometry']]
# Filtrar los nodos de boeing de la zona
zone_blocks_ = gpd.sjoin(blocks_m,zona,predicate='intersects')
zone_blocks_ = zone_blocks_.drop(['index_right'],axis=1)
zone_blocks_.to_file('../data/output/shape/zone_blocks/floresta_HH/floresta_HH_blocks.shp')

In [ ]:
# Zona de analisis
zona = gpd.read_file('../data/input/shape/Poligonos/Bajo_Alta.shp')
zona = zona.to_crs('epsg:4326')
zona = zona[['geometry']]
# Filtrar los nodos de boeing de la zona
zone_blocks_ = gpd.sjoin(blocks_m,zona,predicate='intersects')
zone_blocks_ = zone_blocks_.drop(['index_right'],axis=1)
zone_blocks_.to_file('../data/output/shape/zone_blocks/moravia_LH/moravia_LH_blocks.shp')

In [ ]:
# Zona de analisis
zona = gpd.read_file('../data/input/shape/Poligonos/Alto_Baja.shp')
zona = zona.to_crs('epsg:4326')
zona = zona[['geometry']]
# Filtrar los nodos de boeing de la zona
zone_blocks_ = gpd.sjoin(blocks_m,zona,predicate='intersects')
zone_blocks_ = zone_blocks_.drop(['index_right'],axis=1)
zone_blocks_.to_file('../data/output/shape/zone_blocks/aguacatala_HL/aguacatala_HL_blocks.shp')

# Guadalajara

La selección de las manzanas de las zonas de Guadalajara se hizo de forma manual a partir del archivo "Manzanas_polígonos.gpkg" guardado en la carpeta 03_Datos/03_1_Input/shape/Poligonos_manzanas/

In [2]:
ho = ['miramar','colinas','providencia']
zo = ['miramar_LH','colinas_HL','providencia_HH']
# z = zo[0]

Arreglar el nombre cambiando el input y el conjunto zo a partir de la clasificacion que va a mandar Edgar

In [4]:
# Sacar las capas del geopackage y asignarles el crs de guadalajara
for i in range(len(zo)):
    zone_blocks = gpd.read_file(f'../data/input/shape/zone_blocks_guadalajara/{ho[i]}.shp')
    zone_blocks = zone_blocks.set_crs('epsg:32613', allow_override=True)
    zone_blocks = zone_blocks.to_crs('epsg:4326')
    zone_blocks.to_file(f'../data/output/shape/zone_blocks/{zo[i]}/{zo[i]}_blocks.shp')

Sin embargo, para las manzanas de toda el área urbana será necesario hacer un filtro. Las manzanas de toda el área urbana están guardadas en el archivo "Manzanas_AMG_INEGI20_UTM_v1.shp" en la carpeta 03_Datos/03_1_Input/shape/Manzanas_Guadalajara/. Este archivo contiene unas unas manzanas llamandas contenedoras, las cuales (como su nombre lo dice) contienen a otras manzanas pero estas no representan manzanas verdaderas, por lo tanto será eliminadas para que en la creación de la malla vial por medio de Teselaciones no haya inconvenientes.

In [9]:
blocks = gpd.read_file('../data/input/shape/Manzanas_Guadalajara/Manzanas_AMG_INEGI20_UTM_v1.shp')

In [10]:
print(blocks.crs)

EPSG:32613


In [11]:
# Eliminate 'contenedora' blocks
blocks = blocks[blocks['TIPOMZA']!='Contenedora']
blocks = blocks.to_crs('epsg:4326')
blocks.to_file('../data/output/shape/blocks/guadalajara_blocks.shp')